In [ ]:
import json
import weave
import dataclasses
weave.use_frontend_devmode()
from weave.ops_domain import trace_tree
from weave.wandb_interface.wandb_stream_table import StreamTable

In [ ]:
#stream = StreamTable('shawn/mystream/coolstream3')
stream = StreamTable('shawn/weave-trace-test/test8')

In [ ]:
span = trace_tree.Span(
    _name='span1',
    start_time_ms=0, end_time_ms=100,
    attributes={'hello': 6},
    span_kind=trace_tree.SpanKind.LLM,
    child_spans=[
        trace_tree.Span(
            _name='span2', 
            start_time_ms=10,
            end_time_ms=200,
            span_kind=trace_tree.SpanKind.TOOL,
            attributes={'hello': 9}
        )]
)

In [ ]:
tt = trace_tree.WBTraceTree(json.dumps(dataclasses.asdict(span)))

In [ ]:
stream.log({'a': 50, 'trace': tt})
stream.log({'a': 50, 'b': 100, 'trace': tt})

In [ ]:
from weave import weave_internal

# Had to do this gnarly hack to get the correct type for active_data()
# this happens because Group doesn't correctly hoist granchild variables, so there
# is still a var in the graph when we call table.active_data(), and refine doesn't
# happen in python when there are vars
# TODO: Fix with correct hoisting (at least I think that's the issue)
def get_trace(table):
    active = table.active_data()
    active.type = stream.rows().type
    return active['trace']

board = weave.panels.Board(
    vars={
        'data': stream.rows(),
    },
    panels=[
        weave.panels.BoardPanel(
            # TODO: have to compute all columns we want here, there's no way to
            # tell js or python to do auto-columns behavior.
            lambda data: weave.panels.Table(data,
                                            columns=[
                                                lambda row: row['_timestamp'],
                                                lambda row: row['span_count'],
                                                lambda row: row['duration_ms'],
                                                lambda row: row['request_size'],
                                            ]
                                           ),
            id='table',
            layout=weave.panels.BoardPanelLayout(x=0, y=0, w=24, h=6)
        ),
        weave.panels.BoardPanel(
            lambda table: get_trace(table),
            layout=weave.panels.BoardPanelLayout(x=0, y=6, w=24, h=6)
        )
    ]
)
board

In [ ]:
StreamTable('shawn/weave-trace-test2/test7').rows()